In [36]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import os

In [37]:
LINK_PREFIX = "https://docs.manim.community/en/stable/"
OUTPUT_DIR = "downloaded_media"
INPUT_DIR = "page_content"

In [38]:
def link_to_file_name(link):
    return link[39:].replace('/', '.')

In [39]:
def extract_text_and_media(path):
    with open(path, "rb") as f:
        data = f.read()
    soup = BeautifulSoup(data, "html.parser")
    media_tags = soup.find_all(["img", "video"])
    marker = "<<<MEDIA>>>"
    media_list = []

    # Get images and replace all of them with marker
    for tag in media_tags:
        if tag.name == "img":
            src = tag.get("src") or tag.get("data‑src")
            media_list.append(("image", src))
        else:
            src = tag.get("src")
            if not src and tag.find("source"):
                src = tag.find("source").get("src")
            media_list.append(("video", src))
        tag.replace_with(marker)

    # Get all the texts and split based on marker
    full_text = soup.get_text()
    parts = [piece.strip() for piece in full_text.split(marker)]

    # Interleave
    result = []
    for i, media in enumerate(media_list):
        if parts[i]:
            result.append(("text", parts[i]))
        result.append(media)
    if len(parts) > len(media_list) and parts[-1]:
        result.append(("text", parts[-1]))

    return result

In [40]:
def save_images_and_videos(url, content):
    session = requests.Session()

    for t, info in content:
        if t == "image" or t == "video":
            full_url = urljoin(url, info)
            response = session.get(full_url)

            if response.status_code == 200:
                with open(f"{OUTPUT_DIR}/{link_to_file_name(full_url)}", "wb") as f:
                    f.write(response.content)
                print(f"[saved] {link_to_file_name(full_url)}")
            else:
                print(f"[failed] {full_url} (status code {response.status_code})")

In [41]:
# Save all images and videos
for file_name in os.listdir(INPUT_DIR):
    path = os.path.join(INPUT_DIR, file_name)
    
    with open(path, "rb") as f:
        url = f.readline().decode('utf-8').strip()
    
    content = extract_text_and_media(path)
    save_images_and_videos(url, content)

[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[saved] _static.manim-logo-sidebar.svg
[saved] _static.manim-logo-sidebar-dark.svg
[sav

In [ ]:
def construct_message(content):
    